In [1]:
import pprint
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import webbrowser
import requests
import json

In [2]:
client_id = "enter client id"
client_secret = "enter client secret"
redirect_uri = "https://localhost:8080"
scope = 'playlist-modify-public'

In [1]:
def YouTubeData(url):
    url = url
    response = requests.get(url)
    data = response.json()
    songs = data['items']
    snips = []
    titles = []
    artists = []
    names = []
    for i in songs:
        snip = i['snippet']
        snips.append(snip)
    
    for i in snips:
        title = i['title']
        titles.append(title)
    str(titles)

    for title in titles:
        artist = title.split(' - ')[0]
        artists.append(artist)
    str(artists)   

    for title in titles:
        name = title.split(' - ')[-1]
        names.append(name)
    return artists, names

In [4]:
def FinalList(artists, names):
    fixed = []
    for name in names:
        fix = name.split(' (')[0]
        fixed.append(fix)
    fixed2 = []
    for fix in fixed:
        fix2 = fix.split(' [')[0]
        fix3 = fix2.split(' ft.')[0]
        fix4 = fix3.split('Official')[0]
        fix5 = fix4.split('feat')[0]
        fix6 = fix5.split('Live Session')[0]
        fixed2.append(fix6)
    fixedartists = []
    for artist in artists:
        fix = artist.split(' ')[0]
        fixedartists.append(fix)
    zipped = zip(fixedartists, fixed2)
    zip_list = list(zipped)
    zip_list2 = []

    for i in zip_list:
        fixed = ' '.join(i)
        zip_list2.append(fixed)
    return zip_list2

In [5]:
def create_spotify_playlist(username, playlist_name):
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        playlists = sp.user_playlist_create(username, playlist_name)
        
        find_id = playlists['uri']
        
        print("Playlist Created!")
    
    else:
        print('Cant get token for', username)
    
    return find_id

In [6]:
def get_song_id(final):
    song_id_list = []
    for i in final:
        try: 
            add_tracks = i
            print(add_tracks)
            sp = spotipy.Spotify(auth=token)
            sp.trace = False
            song = sp.search(add_tracks, limit=50, offset=0, type='track', market=None)

            song_search = song['tracks']['items'][0]

            song_id = song_search['id']

            song_id_list.append(song_id)

        except IndexError:
            print("Cant find that song")
            continue 
    print(song_id_list)
    return song_id_list

In [7]:
def add_songs_spotify(username, playlist_id, tracks, position = None):
        if token:
            sp = spotipy.Spotify(auth=token)
            sp.trace = False
            add_tracks = sp.user_playlist_add_tracks(username, playlist_id, tracks, position = None)
            print("Tracks added to playlist!")
        else:
            print('Cant get token for', username)

In [8]:
def get_playlist_link(username, playlist_id):
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        playlist_data = sp.user_playlist(username, playlist_id, fields = all)
        playlist_url = playlist_data['external_urls']['spotify']
        webbrowser.open(playlist_url)
        return playlist_url
    else:
        print('Cant get token for', username)

In [ ]:
playlist_id = str(input("Enter a playlist url: "))
plid = playlist_id.split("list=")[1]
url = "https://www.googleapis.com/youtube/v3/playlistItems?part=snippet,contentDetails&playlistId=" + plid + "key"
data = YouTubeData(url)

In [ ]:
final = FinalList(data[0], data[1])
print(final)

In [ ]:
username = input("Enter your Spotify username: ")

In [ ]:
token = util.prompt_for_user_token(username = username, scope = scope, client_id = client_id, 
                           client_secret = client_secret, redirect_uri = redirect_uri)

print(token)

In [ ]:
playlist_name = input("Enter a name for your playlist: ")

playlist_id = create_spotify_playlist(username, playlist_name)

In [ ]:
song_ids = get_song_id(final)

In [ ]:
tracks = song_ids
add_songs_spotify(username, playlist_id, tracks, position = None)

In [ ]:
get_playlist_link(username, playlist_id)